In [1]:
%config InlineBackend.figure_format = 'retina'

In [2]:
%load_ext autoreload

%autoreload 1
%aimport ds_tutorial.datasets
%aimport ds_tutorial.transformers

In [3]:
import numpy as np
import pandas as pd

import pickle

from pathlib import Path

# Load data

In [4]:
data_root = Path.home() / "data" / "tmp"
reuters_dir = data_root / "reuters21578"
reuters_corpus_path = reuters_dir / "corpus.pkl"
reuters = pickle.load(open(reuters_corpus_path, "rb"))
top_ten_ids, top_ten_names = reuters.top_n(n=10)

cache_dir = reuters_dir / "cache"

In [5]:
cat_ids, cat_names = reuters.top_n(n=90)
label_lookup = {k: v for k, v in zip(cat_ids, cat_names)}

In [6]:
label_lookup[6]

'iron-steel'

In [7]:
topic_lookup = {v: k for k, v in reuters.topics.items()}

## Build dataframe

In [8]:
df, top_ten_ids, train_labels, test_labels = reuters.build_dataframe()

In [9]:
df.head()

,modapte,category,label,date,title,dateline,body,newid,wd_name
0,train,"[interest, retail, ipi]","[0, 1, 2]",1987-03-11 18:14:49,U.S. ECONOMIC DATA KEY TO DEBT FUTURES OUTLOOK,"CHICAGO, March 11 -",U.S. economic data this week could be\nthe key...,4005,Wednesday
1,train,[earn],[3],1987-03-11 18:36:05,BANK OF BRITISH COLUMBIA 1ST QTR JAN 31 NET,"VANCOUVER, British Columbia, March 11 -\n",Oper shr loss two cts vs profit three cts\n ...,4012,Wednesday
2,train,[earn],[3],1987-03-11 18:38:02,RESTAURANT ASSOCIATES INC <RA> 4TH QTR JAN 3,"NEW YORK, March 11 -\n",Shr 25 cts vs 36 cts\n Net 1.4 mln vs 1.4 m...,4014,Wednesday
3,train,[earn],[3],1987-03-11 18:41:59,MICHIGAN GENERAL CORP <MGL> 4TH QTR,"SADDLE BROOK, N.J., March 11 -\n",Shr loss 1.02 dlrs vs 1.01 dlr\n Net loss 1...,4015,Wednesday
4,train,"[crude, nat-gas, iron-steel]","[4, 5, 6]",1987-03-11 18:45:36,"USX <X> PROVED OIL, GAS RESERVES FALL IN 1986","NEW YORK, March 11 -",USX Corp said proved reserves of oil\nand natu...,4016,Wednesday


# Build feature extraction pipeline

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.multiclass import OneVsRestClassifier

from sklearn.feature_selection import SelectKBest, chi2

import xgboost as xgb
import lightgbm as lgb

In [11]:
from ds_tutorial.transformers import TextFromColumns, TextStats, ColumnSelector, TextFromColumns2

In [13]:
df_train = df.query("modapte == 'train'")
df_test = df.query("modapte == 'test'")
y_train = MultiLabelBinarizer().fit_transform(df_train.label.values)
y_test = MultiLabelBinarizer().fit_transform(df_test.label.values)

In [27]:
pipeline = Pipeline(
    memory=str(cache_dir),
    steps=[
        ("union", FeatureUnion(n_jobs=1, transformer_list=[
            ("title_stats", Pipeline([
                ("column", ColumnSelector("title")),
                ("stats", TextStats()),
                #("scaled", StandardScaler()),
            ])),
            ("body_stats", Pipeline([
                ("column", ColumnSelector("body")),
                ("stats", TextStats()),
                #("scaled", StandardScaler()),
            ])),
            ("combined_text", Pipeline([
                ("column", TextFromColumns2()),
                ("tfidf", TfidfVectorizer()),
            ])),
        ])),
        # ("feature_selection", SelectKBest(k=20000)),
        # ("clf", OneVsRestClassifier(LinearSVC(C=1.5))),
        ("clf", OneVsRestClassifier(LogisticRegression(C=100))),
        # ("clf", OneVsRestClassifier(RandomForestClassifier())),
        # ("clf", OneVsRestClassifier(xgb.XGBClassifier())),
])

In [24]:
pipeline.fit(df_train, y_train)
y_pred = pipeline.predict(df_test)
print(classification_report(y_test, y_pred, target_names=top_ten_names, labels=top_ten_ids, digits=3))

/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

        earn      0.972     0.982     0.977      1087
         acq      0.975     0.929     0.952       719
    money-fx      0.786     0.737     0.761       179
       grain      0.969     0.826     0.891       149
       crude      0.911     0.815     0.860       189
       trade      0.835     0.692     0.757       117
    interest      0.890     0.679     0.771       131
        ship      0.941     0.539     0.686        89
       wheat      0.875     0.789     0.830        71
        corn      0.938     0.804     0.865        56

   micro avg      0.945     0.884     0.913      2787
   macro avg      0.909     0.779     0.835      2787
weighted avg      0.943     0.884     0.910      2787
 samples avg      0.759     0.758     0.755      2787



/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [29]:
#pipeline.fit(df_train, y_train)
#y_pred = pipeline.predict(df_test)
#print(classification_report(y_test, y_pred, target_names=top_ten_names, labels=top_ten_ids, digits=3))

# Grid search

In [30]:
from sklearn.model_selection import GridSearchCV

## linear models

In [45]:
param_grid = {
    "clf__estimator__C": [1, 100],
    "clf": [
        OneVsRestClassifier(LinearSVC()),
        OneVsRestClassifier(LogisticRegression())
    ],
}

In [46]:
grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10, scoring="f1_micro")
grid_search.fit(df_train, y_train)
print(grid_search.best_estimator_)

/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf=OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None), clf__estimator__C=1 


/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/pipeline.py:230: UserWarning: Persisting input arguments took 2.71s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jochen/minic

/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklear

[CV]  clf=OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None), clf__estimator__C=1, score=0.821662642290445, total=  39.8s
[CV] clf=OneVsRestClassifier(estimator=LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None), clf__estimator__C=1 


...

[CV]  clf=OneVsRestClassifier(estimator=LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None), clf__estimator__C=100, score=0.8600170502983803, total=  50.3s


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  7.4min finished
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning

/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-package

/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-package

Pipeline(memory='/Users/jochen/data/tmp/reuters21578/cache',
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('title_stats', Pipeline(memory=None,
     steps=[('column', ColumnSelector(column='title', filter_none=True)), ('stats', TextStats()), ('scaled', StandardScaler(copy=True, with_mean=True, with_std=True))])), ('body_stats', Pipeline(me...te=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None))])


In [47]:
print(grid_search.best_estimator_.steps[-1])

('clf', OneVsRestClassifier(estimator=LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None))


In [48]:
print(grid_search.cv_results_["split1_test_score"])

[0.85020101 0.85183958 0.72446556 0.85954094]


In [26]:
print(grid_search.cv_results_["split1_test_score"])

[0.89369927 0.88828759 0.81870893 0.88828759]


In [50]:
y_pred = grid_search.predict(df_test)
print(classification_report(y_test, y_pred, target_names=top_ten_names, labels=top_ten_ids, digits=3))

              precision    recall  f1-score   support

        earn      0.985     0.982     0.983      1087
         acq      0.973     0.946     0.959       719
    money-fx      0.760     0.709     0.734       179
       grain      0.955     0.846     0.897       149
       crude      0.881     0.820     0.849       189
       trade      0.761     0.709     0.735       117
    interest      0.860     0.656     0.745       131
        ship      0.909     0.674     0.774        89
       wheat      0.873     0.775     0.821        71
        corn      0.882     0.804     0.841        56

   micro avg      0.939     0.891     0.914      2787
   macro avg      0.884     0.792     0.834      2787
weighted avg      0.936     0.891     0.912      2787
 samples avg      0.764     0.764     0.760      2787



/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [51]:
#grid_search.cv_results_

## xgboost

In [52]:
import xgboost as xgb

In [53]:
pipeline = Pipeline(
    memory=str(cache_dir),
    steps=[
        ("union", FeatureUnion(n_jobs=1, transformer_list=[
            ("title_stats", Pipeline([
                ("column", ColumnSelector("title")),
                ("stats", TextStats()),
                ("scaled", StandardScaler()),
            ])),
            ("body_stats", Pipeline([
                ("column", ColumnSelector("body")),
                ("stats", TextStats()),
                ("scaled", StandardScaler()),
            ])),
            ("combined_text", Pipeline([
                ("column", TextFromColumns2()),
                ("tfidf", TfidfVectorizer()),
                ("svd", TruncatedSVD(n_components=300, random_state=2018))
            ])),
        ])),
        ("clf", OneVsRestClassifier(xgb.XGBClassifier(nthread=4, early_stopping_rounds=10)))
])

In [44]:
%%time
pipeline.fit(df_train, y_train)
y_pred = pipeline.predict(df_test)
print(classification_report(y_test, y_pred, target_names=top_ten_names, labels=top_ten_ids, digits=3))

              precision    recall  f1-score   support

        earn      0.982     0.980     0.981      1087
         acq      0.956     0.917     0.936       719
    money-fx      0.744     0.665     0.702       179
       grain      0.936     0.691     0.795       149
       crude      0.882     0.794     0.836       189
       trade      0.848     0.667     0.746       117
    interest      0.873     0.473     0.614       131
        ship      0.933     0.315     0.471        89
       wheat      0.940     0.662     0.777        71
        corn      0.944     0.607     0.739        56

   micro avg      0.941     0.841     0.888      2787
   macro avg      0.904     0.677     0.760      2787
weighted avg      0.937     0.841     0.878      2787
 samples avg      0.737     0.730     0.731      2787

CPU times: user 18min 49s, sys: 4.12 s, total: 18min 53s
Wall time: 4min 46s


/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [23]:
print(pipeline.steps[-1])

('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=10, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=4, objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1))


In [54]:
xgb_params = {
    "nthread": [4],
    "objective": ["multi:softprob"],
    "learning_rate": [0.05, 0.001],
    "max_depth": [10],
    "random_state": [2018],
    "gamma": [0, 0.1],
    "reg_alpha": [0],
    "n_estimators": [200],
}

param_grid = {
    "clf": [OneVsRestClassifier(xgb.XGBClassifier(early_stopping_rounds=3))]
}
param_grid.update({f"clf__estimator__{k}": v for k, v in xgb_params.items()})

In [55]:
param_grid

{'clf': [OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
         colsample_bytree=1, early_stopping_rounds=3, gamma=0,
         learning_rate=0.1, max_delta_step=0, max_depth=3,
         min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
         nthread=None, objective='binary:logistic', random_state=0,
         reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
         silent=True, subsample=1),
            n_jobs=None)],
 'clf__estimator__nthread': [4],
 'clf__estimator__objective': ['multi:softprob'],
 'clf__estimator__learning_rate': [0.05, 0.001],
 'clf__estimator__max_depth': [10],
 'clf__estimator__random_state': [2018],
 'clf__estimator__gamma': [0, 0.1],
 'clf__estimator__reg_alpha': [0],
 'clf__estimator__n_estimators': [200]}

In [33]:
%%time
clf = GridSearchCV(pipeline, param_grid, n_jobs=1, 
                   scoring='f1_micro',
                   verbose=2, refit=True)
clf.fit(df_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1), clf__gamma=0, clf__learning_rate=0.05, clf__max_depth=10, clf__n_estimators=200, clf__nthread=4, clf__objective=multi:softprob, clf__random_state=2018, clf__reg_alpha=0 


/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.5min remaining:    0.0s


[CV]  clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1), clf__gamma=0, clf__learning_rate=0.05, clf__max_depth=10, clf__n_estimators=200, clf__nthread=4, clf__objective=multi:softprob, clf__random_state=2018, clf__reg_alpha=0, total= 6.3min
[CV] clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=4, objective='multi:softprob', random_state=2018,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1,

/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=4, objective='multi:softprob', random_state=2018,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1), clf__gamma=0, clf__learning_rate=0.05, clf__max_depth=10, clf__n_estimators=200, clf__nthread=4, clf__objective=multi:softprob, clf__random_state=2018, clf__reg_alpha=0, total= 6.9min
[CV] clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=4, objective='multi:softprob', random_state=2018,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1

...

[CV]  clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0.1,
       learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=4, objective='multi:softprob', random_state=2018,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1), clf__gamma=0.1, clf__learning_rate=0.001, clf__max_depth=10, clf__n_estimators=200, clf__nthread=4, clf__objective=multi:softprob, clf__random_state=2018, clf__reg_alpha=0, total=10.2min
[CV] clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0.1,
       learning_rate=0.001, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=4, objective='multi:softprob', random_state=2018,
       reg_alpha=0, reg_lambda=1, scale_pos_

/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0.1,
       learning_rate=0.001, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=4, objective='multi:softprob', random_state=2018,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1), clf__gamma=0.1, clf__learning_rate=0.001, clf__max_depth=10, clf__n_estimators=200, clf__nthread=4, clf__objective=multi:softprob, clf__random_state=2018, clf__reg_alpha=0, total=11.0min
[CV] clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0.1,
       learning_rate=0.001, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=4, objective='multi:softprob', random_state=2018,
       reg_alpha=0, reg_lambda=1, scale_pos

/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 107.2min finished


[CV]  clf=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=3, gamma=0.1,
       learning_rate=0.001, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=4, objective='multi:softprob', random_state=2018,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1), clf__gamma=0.1, clf__learning_rate=0.001, clf__max_depth=10, clf__n_estimators=200, clf__nthread=4, clf__objective=multi:softprob, clf__random_state=2018, clf__reg_alpha=0, total=11.6min
CPU times: user 7h 41min 3s, sys: 1min 30s, total: 7h 42min 33s
Wall time: 1h 59min 5s


In [34]:
y_pred = clf.predict(df_test)
print(classification_report(y_test, y_pred, target_names=top_ten_names, labels=top_ten_ids, digits=3))

             precision    recall  f1-score   support

       earn      0.956     0.987     0.971      1087
        acq      0.905     0.963     0.933       710
   money-fx      0.644     0.834     0.727       145
      grain      0.421     0.381     0.400        42
      crude      0.711     0.841     0.771       164
      trade      0.672     0.826     0.741       109
   interest      0.710     0.650     0.679       117
       ship      0.600     0.592     0.596        71
      wheat      0.739     0.618     0.673        55
       corn      0.508     0.689     0.585        45

avg / total      0.853     0.906     0.877      2545



/Users/jochen/miniconda3/envs/ds_tutorial/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [37]:
clf.best_estimator_.steps[-1]

('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=1, early_stopping_rounds=3, gamma=0.1,
        learning_rate=0.05, max_delta_step=0, max_depth=10,
        min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
        nthread=4, objective='multi:softprob', random_state=2018,
        reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
        silent=True, subsample=1))